# Generador de personas aleatorias

In [207]:
import random as rm
import numpy as np
import pandas as pd
from faker import Faker
import datetime as dt
import pytz
import re
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from funciones import quitar_acentos
import urllib.parse
from typing import Union

In [ ]:
# TODO agregar lada a municipios y ver si se puede crear un catalogo de puros estados

In [211]:
def sacar_curp(nombre: str, apellido_paterno: str, apellido_materno: str, sexo: str, fecha_nac: Union[str, dt.datetime], estado_nac: str) -> str:
	nombre = quitar_acentos(nombre).upper().replace('Ñ', 'X')
	apellido_paterno = quitar_acentos(apellido_paterno).upper().replace('Ñ', 'X')
	apellido_materno = quitar_acentos(apellido_materno).upper().replace('Ñ', 'X')
	estado_nac = quitar_acentos(estado_nac.replace(' ', '_').lower())
	if isinstance(fecha_nac, str): fecha_nac = dt.datetime.strptime(fecha_nac, '%Y-%m-%d')
	# Primera letra y vocal del primer apellido
	curp = apellido_paterno[0:2].upper()
	# Primera letra del segundo apellido
	curp += apellido_materno[0:1].upper()
	# Primera letra del nombre de pila
	curp += nombre[0:1].upper()
	# Fecha de nacimiento (2 últimos dígitos del año, 2 del mes y 2 del día de nacimiento)
	curp += fecha_nac.strftime('%y%m%d')
	# Letra del sexo (H o M)
	curp += sexo[0:1].upper()
	# Dos letras correspondientes a la entidad de nacimiento segun RENAPO (Sonora => SR)
	# en el caso de extranjeros, se marca como NE (Nacido Extranjero)
	clave_estados = {
		'aguascalientes': 'AS', 'baja_california': 'BC', 'baja_california_sur': 'BS',
		'campeche': 'CC', 'coahuila': 'CL', 'colima': 'CM', 'chiapas': 'CS', 'chihuahua': 'CH',
		'distrito_federal': 'DF', 'ciudad_de_mexico': 'DF', 'durango': 'DG', 'guanajuato': 'GT',
		'guerrero': 'GR', 'hidalgo': 'HG', 'jalisco': 'JC', 'mexico': 'MC', 'estado_de_mexico': 'MC',
		'michoacan': 'MN', 'morelos': 'MS', 'nayarit': 'NT', 'nuevo_leon': 'NL', 'oaxaca': 'OC',
		'puebla': 'PL', 'queretaro': 'QT', 'quintana_roo': 'QR', 'san_luis_potosi': 'SP', 'sinaloa': 'SL',
		'sonora': 'SR', 'tabasco': 'TC', 'tamaulipas': 'TS', 'tlaxcala': 'TL', 'veracruz': 'VZ',
		'yucatan': 'YN', 'zacatecas': 'ZS'
	}
	if estado_nac in clave_estados:
		clave_estado_nac = clave_estados[estado_nac]
	else:
		clave_estado_nac = 'NE'
	curp += clave_estado_nac
	# Primera consonante interna del primer apellido
	curp += re.sub(f'[aeiou]', '', apellido_paterno)[1:2].upper()
	# Primera consonante interna del segundo apellido
	curp += re.sub(f'[aeiou]', '', apellido_materno)[1:2].upper()
	# Primera consonante interna del nombre
	curp += re.sub(f'[aeiou]', '', nombre)[1:2].upper()
	# Dígito verificador del 0-9 para fechas de nacimiento hasta el año 1999 y A-Z para fechas de nacimiento a partir del 2000
	int_char = [rm.randint(0, 9), chr(rm.randint(65, 90))]
	curp += str(int_char[0] if fecha_nac.year <= 1999 else int_char[1])
	# Homoclave, para evitar duplicaciones
	curp += str(rm.randint(0, 9))
	return curp

def obtener_coordenadas(ciudad, estado, pais = 'Mexico'):
	# Hacer peticion a un api para obtener coordenadas del centro y extremas
	# querystring = urllib.parse.urlencode({'city': ciudad, 'state': estado, 'country': pais})
	querystring = urllib.parse.urlencode({'q': f'{ciudad} {estado} {pais}'})
	try:
		df = pd.read_json(f'https://nominatim.openstreetmap.org/search?{querystring}&polygon_geojson=1&format=json')
		# En caso de que el dataframe este vacio
		if df.empty: raise BaseException(f'No se encontraron coordenadas de "{ciudad}", "{estado}", "{pais}"')
		# Obtener coordenadas del centro
		centro = (df['lat'][0], df['lon'][0])
		# Obtener extremos originales
		minx, maxx, miny, maxy = [float(i) for i in df['boundingbox'][0]]
		ne, no, se, so = [(maxx, maxy), (maxx, miny), (minx, maxy), (minx, miny)]
		print(f'\033[33mobtener_coordenadas()\033[0m: Coordenadas encontradas en "{ciudad}", "{estado}", "{pais}"')
	except BaseException as e:
		print(f'\033[31mobtener_coordenadas()\033[0m:', e)
		return None

	return {'centro': centro, 'noreste': ne, 'noroeste': no, 'sureste': se, 'suroeste': so}

def generar_puntos_random(centro, extremo_so, extremo_ne, n = 1, reducir = True):
	# Obtener coordenadas del centro
	cenx, ceny = centro
	# Obtener extremos originales
	minx, miny = extremo_so
	maxx, maxy = extremo_ne
	# Hallar punto medio entre (suroeste centro) y (centro noreste)
	minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
	maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# True para reducir el area de la ciudad
	if reducir:
		# Hallar punto medio de puntos medios y centro
		minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
		maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# Obtener n puntos aleatorios dentro del area
	puntos_random = []
	while len(puntos_random) < n:
		puntos_random.append((rm.uniform(minx, maxx), rm.uniform(miny, maxy)))
	return puntos_random

def buscar_direccion(puntos_random):
	geolocator = Nominatim(user_agent = 'geoapiExercises')
	dirrr = {}
	listo = False
	for punto_random in puntos_random:
		dirr = {}
		direccion = geolocator.reverse(punto_random, language = 'es_MX')
		direccion = direccion[0].split(', ')
		print(direccion)
		if len(direccion) == 10:
			print('\033[34mHay de 10\033[0m')
			print(list(direccion))
		elif len(direccion) == 9:
			print('\033[34mHay de 9\033[0m')
			print(list(direccion))
			# [
			#   'Jardin De Niños Gabino Barrera',
			#   'Avenida Azarco',
			#   'Villa Insurgentes (El Calabazal)',
			#   'Barrio El Cerrito',
			#   'villa insurgentes  (El Calabazal)',
			#   'Sombrerete',
			#   'Zacatecas',
			#   '99109',
			#   'México'
			# ]
			dirr['local'] = direccion[0]
			dirr['calle'] = direccion[1]
			dirr['numero'] = rm.randint(1, 999)
			dirr['colonia'] = direccion[3]
			dirr['cp'] = direccion[-2]
			dirr['ciudad'] = direccion[-4]
			dirr['estado'] = direccion[-3]
			dirr['pais'] = direccion[-1]
			listo = True
		elif len(direccion) == 8:
			dirr['local'] = direccion[0]
			dirr['numero'] = direccion[1]
			dirr['calle'] = direccion[2]
			dirr['colonia'] = direccion[3]
			dirr['cp'] = direccion[-2]
			dirr['ciudad'] = direccion[-4]
			dirr['estado'] = direccion[-3]
			dirr['pais'] = direccion[-1]
			listo = True
		elif len(direccion) == 7:
			dirr['local'] = direccion[0]
			dirr['calle'] = direccion[1]
			dirr['numero'] = rm.randint(1, 999)
			dirr['colonia'] = direccion[2]
			dirr['cp'] = direccion[-2]
			dirr['ciudad'] = direccion[-4]
			dirr['estado'] = direccion[-3]
			dirr['pais'] = direccion[-1]
			listo = True
		elif len(direccion) == 6:
			dirr['calle'] = direccion[0]
			dirr['numero'] = rm.randint(1, 999)
			dirr['colonia'] = direccion[1]
			dirr['cp'] = direccion[-2] if direccion[-2].isnumeric() else str(rm.randint(80000, 99999))
			dirr['ciudad'] = direccion[-4]
			dirr['estado'] = direccion[-3]
			dirr['pais'] = direccion[-1]
			listo = True
		elif len(direccion) == 5:
			dirr['calle'] = direccion[0]
			dirr['numero'] = rm.randint(1, 999)
			dirr['colonia'] = direccion[1]
			dirr['ciudad'] = direccion[-3]
			if direccion[-2].isnumeric():
				dirr['cp'] = direccion[-2]
				dirr['estado'] = direccion[-3]
			else:
				dirr['cp'] = str(rm.randint(80000, 99999))
				dirr['estado'] = direccion[-2]
			dirr['pais'] = direccion[-1]
			listo = True
		else:
			continue
		if listo:
			print('\033[33mbuscar_direccion()\033[0m: Direccion encontrada en', punto_random)
			dirrr = dirr
			break
	if not listo:
		print('\033[33mbuscar_direccion()\033[0m: No se encontro una direccion')
	return dirrr

In [212]:
reap = pd.read_csv('./conjunto_datos/relacion_edad_altura_peso.csv')
nombres = pd.read_csv('./catalogos/nombres.csv')
profesiones = pd.read_csv('./catalogos/profesiones.csv')
file_ciudades = './catalogos/ciudades.csv'
ct_ciudades = pd.read_csv(file_ciudades)

def personas_aleatorias(n = 5):
	fake = Faker(['es_MX'])
	personas = []
	for _ in range(n):
		print('Idx', _)
		sexo = rm.choice(['H', 'M'])
		# generar tipo de sangre y asignar probabilidad a cada valor para que salga
		tipo_sangre = rm.choices(['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-'], weights = [30, 6, 9, 2, 4, 1, 39, 9])
		tipo_sangre = tipo_sangre[0]

		fecha_nac = fake.date_of_birth(minimum_age = int(reap['edad'].min()))
		# calcular edad en base a la fecha de nacimiento
		dia_actual = dt.date.today()
		edad = dia_actual.year - fecha_nac.year - ((dia_actual.month, dia_actual.day) < (fecha_nac.month, fecha_nac.day))

		# dependiendo de la edad se busca el margen de alturas y pesos que tenga esa edad
		# si es mayor que la edad maxima que este registrada se le da el margen de esa edad maxima
		# mucha gente deja de crecer a los 20
		margen_ap = reap[(reap['edad'] == edad) if edad <= reap['edad'].max() else (reap['edad'] == reap['edad'].max())].reset_index()

		# dependiendo del sexo se generan nombres femeninos o masculinos
		# y estaturas y pesos dentro de su margen correspondiente
		if sexo == 'H':
			titulo = rm.choice(['Sr.', 'Dr.', 'Mtro.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['h'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.h.cm'][0], margen_ap['lim_sup.altura.h.cm'][0]), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.h.kg'][0], margen_ap['lim_sup.peso.h.kg'][0]), 2)
		else:
			titulo = rm.choice(['Sra.', 'Dra.', 'Mtra.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['m'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.m.cm'][0], margen_ap['lim_sup.altura.m.cm'][0]), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.m.kg'][0], margen_ap['lim_sup.peso.m.kg'][0]), 2)

		apellido_paterno = fake.last_name()
		apellido_materno = fake.last_name()

		id_estado = rm.choice(ct_ciudades['id_estado'].unique())
		ct_ciudad = ct_ciudades[ct_ciudades['id_estado'] == id_estado].sample()
		ct_ciudad_index_row = ct_ciudad.index.values[0]
		ct_ciudad = ct_ciudad.to_dict(orient = 'records')[0]

		ciudad = ct_ciudad['ciudad']
		estado = ct_ciudad['estado']
		tel = f'({ct_ciudad["clave_lada"]})' + str(rm.randint(1000000, 9999999))
		curp = sacar_curp(nombre, apellido_paterno, apellido_materno, sexo, fecha_nac, estado)

		# Generar un nombre de usuario para un email
		switcher = {
			1: fake.user_name(),
			# aperez
			2: f'{nombre[0]}{apellido_paterno}',
			# alonso_perez
			3: f'{nombre}_{apellido_paterno}',
			# aps70
			4: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.strftime("%y")}',
			# aps1970
			5: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.year}',
			# alonsoperezsoltero
			6: f'{nombre}{apellido_paterno}{apellido_materno}'
		}
		rand_int = rm.randint(min(list(switcher.keys())), max(list(switcher.keys())))
		usuario = switcher.get(rand_int, 'usuario')
		# quitar espacios o reemplazarlos por un _
		usuario = usuario.replace(' ', rm.choice(['', '_'])).lower()
		usuario = quitar_acentos(usuario)
		subdominio = rm.choice(['.com', '.net', '.io', '.gob', '.org', '.edu'])
		correo = usuario + f'@example{subdominio}'

		profesion = profesiones.sample().values[0][0] if edad >= 18 else None
		"""
		El Número de Seguridad Social (NSS) es una homoclave que asigna el IMSS
		para llevar un registro de los trabajadores y asegurados
		que están inscritos en dicha institución.
		El NSS es único, permanente e intransferible.
		"""
		# El NSS está compuesto por 11 dígitos:
		if profesion:
			# Los primeros dos caracteres están vinculados a la subdelegación en el que la persona fue afiliada.
			nss1 = ('%02d' % rm.randint(0, 99))
			# Los dos dígitos siguientes indican el año en el que la persona se afilió al Seguro Social.
			nss2 = ('%02d' % rm.randint(0, 99))
			# Los siguientes dos dígitos corresponden a la fecha de nacimiento del afiliado.
			nss3 = fecha_nac.strftime('%y')
			# Los cuatro números siguientes son los dígitos que asigna el IMSS al trabajador.
			nss4 = ('%02d' % rm.randint(0, 9999))
			# El último dígito corresponde al número de verificación del trabajador en el IMSS.
			nss5 = str(rm.randint(0, 9))
			nss = f'{nss1} {nss2} {nss3} {nss4} {nss5}'
		else:
			nss = None

		"""Coordenadas de la ciudad"""
		centro = suroeste = noreste = None
		# si el catalogo ya tiene coordenadas, tomar esos valores
		if pd.notna(ct_ciudad['centro']) and pd.notna(ct_ciudad['suroeste']) and pd.notna(ct_ciudad['noreste']):
			centro = eval(ct_ciudad['centro'])
			suroeste = eval(ct_ciudad['suroeste'])
			noreste = eval(ct_ciudad['noreste'])
		else:
			# si no, buscar en internet el punto central y extremos segun la ciudad
			try:
				coordenadas = obtener_coordenadas(ciudad, estado)
				if coordenadas:
					centro = coordenadas['centro']
					suroeste = coordenadas['suroeste']
					noreste = coordenadas['noreste']
					# guardar coordenadas en el catalogo
					# para que a la otra no haya que buscar en internet
					ct_ciudades.loc[ct_ciudad_index_row, 'centro'] = str(centro)
					ct_ciudades.loc[ct_ciudad_index_row, 'suroeste'] = str(suroeste)
					ct_ciudades.loc[ct_ciudad_index_row, 'noreste'] = str(noreste)
					ct_ciudades.to_csv(file_ciudades, index = False)
			except BaseException as be:
				# En caso de no haber encontrado o no tener internet
				print(f'\033[31mCoordsException: idx {_}\033[0m')
				print(be)

		"""Direccion"""
		direcc = cp = None
		if centro and suroeste and noreste:
			puntos_random = generar_puntos_random(centro, suroeste, noreste, 5)
			try:
				direccion = buscar_direccion(puntos_random)
				if all(k in direccion for k in ('calle', 'numero', 'colonia', 'cp')):
					calle = direccion['calle']
					numero = direccion['numero']
					colonia = direccion['colonia']
					cp = direccion['cp']
					# print(pd.DataFrame(direccion))
					direcc = f'{calle} #{numero}, {colonia}'
			except BaseException as be:
				print(f'\033[31mAddressException: idx {_}\033[0m')
				print(be)

		"""Zona horaria"""
		timezone = tz_offset = None
		if centro:
			# obtener zona horaria segun latitud y longitud
			obj = TimezoneFinder()
			#if centro[1] > 180.0 or centro[1] < -180.0 or centro[0] > 90.0 or centro[0] < -90.0:
			timezone = obj.timezone_at(lat = centro[0], lng = centro[1])
			# en Enero no hay DST
			#tz_offset = pytz.timezone(timezone).localize(dt.datetime(2011, 1, 1)).strftime('%z')
			# 'datetime.now' tiene DST (Daylight Saving Time)
			tz_offset = dt.datetime.now(pytz.timezone(timezone)).strftime('%z')

		if not centro: centro = (np.nan, np.nan)

		personas.append({
			'titulo': titulo,
			'nombre': nombre,
			'apellido_paterno': apellido_paterno,
			'apellido_materno': apellido_materno,
			'fecha_nac': fecha_nac.strftime('%Y-%m-%d'),
			'sexo': sexo,
			'altura': altura,
			'peso': peso,
			'tipo_sangre': tipo_sangre,
			'correo': correo,
			'tel': tel,
			'direccion': direcc,
			'cp': cp,
			'ciudad': ciudad,
			'estado': estado,
			'lat': centro[0],
			'lon': centro[1],
			'tz_offset': tz_offset,
			'zona_horaria': timezone,
			'profesion': profesion,
			'curp': curp,
			'nss': nss,
			'color_fav': fake.safe_color_name(),
		})
	return pd.DataFrame(personas)

personas_aleatorias(10)

Idx 0
obtener_coordenadas(): Coordenadas encontradas en "Córdoba", "Veracruz", "Mexico"
['Agustín Millán', 'Córdoba', 'Veracruz', '94560', 'México']
buscar_direccion(): Direccion encontrada en (18.906949369309814, -96.91735110920823)
Idx 1
obtener_coordenadas(): Coordenadas encontradas en "San Luis Acatlán", "Guerrero", "Mexico"
['Renacimiento', 'San Luis Acatlán', 'Guerrero', 'México']
['Rancho Josefina Guillén', 'San Luis Acatlán', 'Guerrero', 'México']
['La Raya', 'San Luis Acatlán', 'Guerrero', 'México']
['Yoloxóchitl', 'San Luis Acatlán', 'Guerrero', 'México']
['Atotonilco de Horcasitas', 'San Luis Acatlán', 'Guerrero', 'México']
buscar_direccion(): No se encontro una direccion
Idx 2
obtener_coordenadas(): Coordenadas encontradas en "Yuriria", "Guanajuato", "Mexico"
['Moroleón', 'Yuriria', 'Guanajuato', '38990', 'México']
buscar_direccion(): Direccion encontrada en (20.15334886573289, -101.25865760427683)
Idx 3
obtener_coordenadas(): Coordenadas encontradas en "Puerto Aventuras", 

,titulo,nombre,apellido_paterno,apellido_materno,fecha_nac,sexo,altura,peso,tipo_sangre,correo,...,ciudad,estado,lat,lon,tz_offset,zona_horaria,profesion,curp,nss,color_fav
0,Lic.,Antonia,Ayala,Martínez,1906-12-03,M,154.61,54.08,A+,aam06@example.net,...,Córdoba,Veracruz,18.923971,-96.927811,-0600,America/Mexico_City,Redactor publicitario,AYMA061203MVZYAN05,29 95 06 6185 0,maroon
1,None,Arcelia,Curiel,Gaona,1935-09-08,M,163.58,66.28,O+,arceliacurielgaona@example.io,...,San Luis Acatlán,Guerrero,16.870085,-98.722207,-0600,America/Mexico_City,Administrador de deportes,CUGA350908MGRUAR75,77 33 35 1806 9,white
2,Lic.,Isabel,Muñoz,Soto,1983-08-02,M,172.91,73.91,O-,vanesaaragon@example.gob,...,Yuriria,Guanajuato,20.166492,-101.317134,-0600,America/Mexico_City,Horticultor comercial,MUSI830802MGTUOS55,70 55 83 1186 5,navy
3,None,Timoteo,Montoya,Espino,1992-03-19,H,175.82,80.78,A+,tmontoya@example.io,...,Puerto Aventuras,Quintana Roo,20.506668,-87.230063,-0500,America/Cancun,Tesorero corporativo,MOET920319HQROSI25,26 93 92 5872 2,maroon
4,Sr.,César,Urbina,Escalante,1998-05-21,H,168.96,90.73,O+,cesarurbinaescalante@example.net,...,Dzitbalche,Campeche,20.318590,-90.056539,-0600,America/Merida,Ingeniero de comunicaciones,UREC980521HCCRSE73,74 38 98 5667 8,gray
5,None,María,Villagómez,Almaraz,2015-06-24,M,116.26,25.56,A+,maria_villagomez@example.net,...,Coyoacán,Ciudad de Mexico,19.350094,-99.162332,-0600,America/Mexico_City,None,VIAM150624MDFILAW9,None,teal
6,Dra.,Rosalia,Corral,Calvillo,1944-10-31,M,154.65,84.65,A+,rcc1944@example.gob,...,Villa de Juárez,Baja California,31.967873,-116.650080,-0700,America/Tijuana,Cartógrafo,COCR441031MBCOAO94,05 27 44 7551 1,purple
7,None,José,Loya,Peña,1970-10-03,H,181.67,60.02,A+,joseloyapena@example.net,...,Cuéncame de Ceniceros,Durango,24.862276,-103.685920,-0600,America/Monterrey,Citogenetista,LOPJ701003HDGOEO36,08 34 70 3982 5,teal
8,None,Daniel,Vanegas,Ferrer,2008-08-29,H,155.70,67.18,B+,hugo35@example.net,...,Santiago,Baja California Sur,23.481372,-109.714600,-0700,America/Mazatlan,None,VAFD080829HBSAEAF3,None,aqua
9,Lic.,Aida,Carreón,Carbajal,1945-10-07,M,170.17,55.43,A+,aidacarreoncarbajal@example.com,...,Poblado Héroes de Chapultepec,Baja California,30.979810,-116.164365,-0700,America/Tijuana,Contador Público,CACA451007MBCAAI89,12 31 45 4124 9,silver
